In [8]:
import operator
import random
import time
import math
import numpy
import pandas
from concurrent.futures import ProcessPoolExecutor
import slim

In [1]:
class Data:
    def __init__(self, dataset='ml-100k'):
        """
        无上下文信息的隐性反馈数据集。
        :param dataset: 使用的数据集名字，当前有'ml-100k','ml-1m'
        """

        path = None
        separator = None
        if dataset == 'ml-100k':
            # 共100000条数据，只有943个用户看过电影，只有1682个电影被看过。userID范围[1, 943]，movieID范围[1, 1682]。
            path = '/Users/qtt/workspace/github/SLIM-recommendation/data/ml-100k/u.data'
            separator = '\t'
        elif dataset == 'ml-1m':
            # 共1000209条数据，只有6040个用户看过电影，只有3706个电影被看过。userID范围[1, 6040]，movieID范围[1, 3952]。
            path = '/Users/qtt/workspace/github/SLIM-recommendation/data/ml-1m/ratings.dat'
            separator = '::'

        print('开始读取数据')

        # 从源文件读数据
        self.data = []
        for line in open(path, 'r'):
            data_line = line.split(separator)
            userID = int(data_line[0])
            movieID = int(data_line[1])
            # 无上下文信息的隐性反馈数据不需要评分和时间截
            #rating = int(data_line[2])
            #timestamp = int(data_line[3])
            self.data.append([userID, movieID])

        def compress(data, col):
            """
            压缩数据data第col列的数据。保证此列数字会从0开始连续出现，中间不会有一个不存在此列的数字。
            :param data: 二维列表数据
            :param col: 要压缩的列
            :return: 此列不同的数字个数（即此列最大数字加1）
            """
            e_rows = dict()  # 键是data数据第col列数据，值是一个存放键出现在的每一个行号的列表
            for i in range(len(data)):
                e = data[i][col]
                if e not in e_rows:
                    e_rows[e] = []
                e_rows[e].append(i)

            for rows, i in zip(e_rows.values(), range(len(e_rows))):
                for row in rows:
                    data[row][col] = i

            return len(e_rows)

        self.num_user = compress(self.data, 0)
        self.num_item = compress(self.data, 1)

        # 训练集和测试集
        self.train, self.test = self.__split_data()
        print('总共有{}条数据，训练集{}，测试集{}，用户{}，物品{}'.format(len(self.data), len(self.train), len(self.test), self.num_user, self.num_item))

    def __split_data(self):
        """
        将数据随机分成8份，1份作为测试集，7份作为训练集
        :return: 训练集和测试集
        """
        test = []
        train = []
        for user, item in self.data:
            if random.randint(1, 8) == 1:
                test.append([user, item])
            else:
                train.append([user, item])
        return train, test

In [5]:
data = Data()

开始读取数据
总共有100000条数据，训练集87458，测试集12542，用户943，物品1682


In [11]:

class SLIM:
    def __init__(self, data):
        """
        稀疏线性算法。
        :param data: 无上下文信息的隐性反馈数据集，包括训练集，测试集等
        """
        self.data = data

        print('稀疏线性算法')
        self.A = self.__user_item_matrix()  # 用户-物品行为矩阵

        self.alpha = None
        self.lam_bda = None
        self.max_iter = None  # 学习最大迭代次数
        self.tol = None  # 学习阈值
        self.N = None  # 每个用户最多推荐物品数量
        self.lambda_is_ratio = None  # lambda参数是否代表比例值

        self.W = None  # 系数集合
        self.recommendation = None

    def compute_recommendation(self, alpha=0.5, lam_bda=0.02, max_iter=1000, tol=0.0001, N=10, lambda_is_ratio=True):
        """
        开始计算推荐列表
        :param alpha: lasso占比（为0只有ridge-regression，为1只有lasso）
        :param lam_bda: elastic net系数
        :param max_iter: 学习最大迭代次数
        :param tol: 学习阈值
        :param N: 每个用户最多推荐物品数量
        :param lambda_is_ratio: lambda参数是否代表比例值。若为True，则运算时每列lambda单独计算；若为False，则运算时使用单一lambda的值
        """
        self.alpha = alpha
        self.lam_bda = lam_bda
        self.max_iter = max_iter
        self.tol = tol
        self.N = N
        self.lambda_is_ratio = lambda_is_ratio

        print('开始计算W矩阵（alpha=' + str(self.alpha) + ', lambda=' + str(self.lam_bda) + ', max_iter=' + str(
            self.max_iter) + ', tol=' + str(self.tol) + '）')
        self.W = self.__aggregation_coefficients()

        print('开始计算推荐列表（N=' + str(self.N) + '）')
        self.recommendation = self.__get_recommendation()

    def __user_item_matrix(self):
        A = numpy.zeros((self.data.num_user, self.data.num_item))
        for user, item in self.data.train:
            A[user, item] = 1
        return A

    def __aggregation_coefficients(self):
        group_size = 100  # 并行计算每组计算的行/列数
        n = self.data.num_item // group_size  # 并行计算分组个数
        starts = []
        ends = []
        for i in range(n):
            start = i * group_size
            starts.append(start)
            ends.append(start + group_size)
        if self.data.num_item % group_size != 0:
            starts.append(n * group_size)
            ends.append(self.data.num_item)
            n += 1

        print('进行covariance updates的预算')
        covariance_array = None
        with ProcessPoolExecutor() as executor:
            covariance_array = numpy.vstack(executor.map(slim.compute_covariance, [self.A] * n, starts, ends))
        slim.symmetrize_covariance(covariance_array)

        print('坐标下降法学习W矩阵')
        if self.lambda_is_ratio:
            with ProcessPoolExecutor() as executor:
                return numpy.hstack(executor.map(slim.coordinate_descent_lambda_ratio, [self.alpha] * n, [self.lam_bda] * n, [self.max_iter] * n, [self.tol] * n, [self.data.num_user] * n, [self.data.num_item] * n, [covariance_array] * n, starts, ends))
        else:
            with ProcessPoolExecutor() as executor:
                return numpy.hstack(executor.map(slim.coordinate_descent, [self.alpha] * n, [self.lam_bda] * n, [self.max_iter] * n, [self.tol] * n, [self.data.num_user] * n, [self.data.num_item] * n, [covariance_array] * n, starts, ends))

    def __recommend(self, user_AW, user_item_set):
        """
        给用户user推荐最多N个物品。
        :param user_AW: AW矩阵相乘的第user行
        :param user_item_set: 训练集用户user所有有过正反馈的物品集合
        :return: 推荐给本行用户的物品列表
        """
        rank = dict()
        for i in set(range(self.data.num_item)) - user_item_set:
            rank[i] = user_AW[i]
        return [items[0] for items in sorted(rank.items(), key=operator.itemgetter(1), reverse=True)[:self.N]]

    def __get_recommendation(self):
        """
        得到所有用户的推荐物品列表。
        :return: 推荐列表，下标i对应给用户i推荐的物品列表
        """
        # 得到训练集中每个用户所有有过正反馈物品集合
        train_user_items = [set() for u in range(self.data.num_user)]
        for user, item in self.data.train:
            train_user_items[user].add(item)

        AW = self.A.dot(self.W)

        # 对每个用户推荐最多N个物品
        recommendation = []
        for user_AW, user_item_set in zip(AW, train_user_items):
            recommendation.append(self.__recommend(user_AW, user_item_set))
        return recommendation

In [12]:
class Evaluation:
    def __init__(self, recommend_algorithm):
        """
        对推荐算法recommend_algorithm计算各种评测指标。
        :param recommend_algorithm: 推荐算法，包括推荐结果列表，数据集等
        """
        self.rec_alg = recommend_algorithm

        self.precision = None
        self.recall = None
        self.coverage = None
        self.popularity = None

    def evaluate(self):
        """
        评测指标的计算。
        """
        # 准确率和召回率
        self.precision, self.recall = self.__precision_recall()
        print('准确率 = ' + str(self.precision * 100) + "%  召回率 = " + str(self.recall * 100) + '%')

        # 覆盖率
        self.coverage = self.__coverage()
        print('覆盖率 = ' + str(self.coverage * 100) + '%')

        # 流行度
        self.popularity = self.__popularity()
        print('流行度 = ' + str(self.popularity))

    def __precision_recall(self):
        """
        计算准确率和召回率。
        :return: 准确率和召回率
        """
        # 得到测试集用户与其所有有正反馈物品集合的映射
        test_user_items = dict()
        for user, item in self.rec_alg.data.test:
            if user not in test_user_items:
                test_user_items[user] = set()
            test_user_items[user].add(item)

        # 计算准确率和召回率
        hit = 0
        all_ru = 0
        all_tu = 0
        for user, items in test_user_items.items():
            ru = set(self.rec_alg.recommendation[user])
            tu = items

            hit += len(ru & tu)
            all_ru += len(ru)
            all_tu += len(tu)
        return hit / all_ru, hit / all_tu

    def __coverage(self):
        """
        计算覆盖率
        :return: 覆盖率
        """
        recommend_items = set()
        for user in range(self.rec_alg.data.num_user):
            for item in self.rec_alg.recommendation[user]:
                recommend_items.add(item)
        return len(recommend_items) / self.rec_alg.data.num_item

    def __popularity(self):
        """
        计算新颖度（平均流行度）
        :return: 新颖度
        """
        item_popularity = [0 for i in range(self.rec_alg.data.num_item)]
        for user, item in self.rec_alg.data.train:
            item_popularity[item] += 1

        ret = 0
        n = 0
        for user in range(self.rec_alg.data.num_user):
            for item in self.rec_alg.recommendation[user]:
                ret += math.log(1 + item_popularity[item])
                n += 1
        return ret / n

In [13]:

recommend = SLIM(data)
recommend.compute_recommendation()
eva = Evaluation(recommend)
eva.evaluate()
print(eva.precision)

稀疏线性算法
开始计算W矩阵（alpha=0.5, lambda=0.02, max_iter=1000, tol=0.0001）
进行covariance updates的预算


/Users/qtt/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:69: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


坐标下降法学习W矩阵
column 0 learnt 51 steps
column 1 learnt 126 steps
column 2 learnt 70 steps


/Users/qtt/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:75: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


column 3 learnt 114 steps
column 100 learnt 189 steps
column 4 learnt 74 steps
column 200 learnt 177 steps
column 5 learnt 128 steps
column 101 learnt 338 steps
column 300 learnt 48 steps
column 201 learnt 216 steps
column 102 learnt 210 steps
column 6 learnt 217 steps
column 202 learnt 49 steps
column 301 learnt 401 steps
column 7 learnt 95 steps
column 400 learnt 75 steps
column 103 learnt 188 steps
column 302 learnt 116 steps
column 203 learnt 120 steps
column 8 learnt 122 steps
column 401 learnt 91 steps
column 303 learnt 61 steps
column 104 learnt 109 steps
column 304 learnt 41 steps
column 500 learnt 73 steps
column 9 learnt 108 steps
column 204 learnt 111 steps
column 105 learnt 57 steps
column 402 learnt 350 steps
column 600 learnt 73 steps
column 305 learnt 100 steps
column 10 learnt 100 steps
column 501 learnt 78 steps
column 601 learnt 52 steps
column 205 learnt 91 steps
column 106 learnt 55 steps
column 11 learnt 49 steps
column 700 learnt 47 steps
column 306 learnt 68 step

column 338 learnt 103 steps
column 537 learnt 73 steps
column 638 learnt 38 steps
column 742 learnt 42 steps
column 234 learnt 168 steps
column 141 learnt 153 steps
column 435 learnt 138 steps
column 40 learnt 77 steps
column 339 learnt 68 steps
column 743 learnt 63 steps
column 744 learnt 48 steps
column 235 learnt 74 steps
column 538 learnt 125 steps
column 41 learnt 51 steps
column 340 learnt 71 steps
column 236 learnt 36 steps
column 745 learnt 40 steps
column 539 learnt 39 steps
column 436 learnt 104 steps
column 341 learnt 37 steps
column 142 learnt 141 steps
column 237 learnt 99 steps
column 42 learnt 49 steps
column 540 learnt 55 steps
column 639 learnt 797 steps
column 143 learnt 65 steps
column 746 learnt 102 steps
column 238 learnt 98 steps
column 640 learnt 33 steps
column 541 learnt 79 steps
column 437 learnt 127 steps
column 43 learnt 87 steps
column 641 learnt 37 steps
column 747 learnt 45 steps
column 144 learnt 88 steps
column 342 learnt 184 steps
column 748 learnt 53 

column 785 learnt 533 steps
column 178 learnt 66 steps
column 375 learnt 112 steps
column 467 learnt 58 steps
column 576 learnt 63 steps
column 786 learnt 119 steps
column 677 learnt 79 steps
column 376 learnt 54 steps
column 75 learnt 83 steps
column 577 learnt 102 steps
column 468 learnt 91 steps
column 273 learnt 223 steps
column 787 learnt 90 steps
column 179 learnt 244 steps
column 76 learnt 147 steps
column 578 learnt 41 steps
column 678 learnt 66 steps
column 377 learnt 84 steps
column 180 learnt 39 steps
column 274 learnt 91 steps
column 579 learnt 32 steps
column 378 learnt 51 steps
column 469 learnt 82 steps
column 580 learnt 68 steps
column 379 learnt 36 steps
column 275 learnt 122 steps
column 77 learnt 184 steps
column 181 learnt 136 steps
column 581 learnt 59 steps
column 380 learnt 59 steps
column 470 learnt 72 steps
column 679 learnt 1000 steps
column 381 learnt 40 steps
column 788 learnt 1000 steps
column 582 learnt 44 steps
column 78 learnt 121 steps
column 276 learnt

column 1119 learnt 56 steps
column 1020 learnt 66 steps
column 918 learnt 251 steps
column 499 learnt 63 steps
column 1120 learnt 37 steps
column 1021 learnt 47 steps
column 829 learnt 64 steps
column 1211 learnt 41 steps
column 919 learnt 40 steps
column 1402 learnt 394 steps
column 1121 learnt 70 steps
column 830 learnt 49 steps
column 1022 learnt 47 steps
column 920 learnt 69 steps
column 921 learnt 42 steps
column 831 learnt 78 steps
column 1308 learnt 1000 steps
column 1023 learnt 69 steps
column 922 learnt 44 steps
column 1122 learnt 504 steps
column 1309 learnt 52 steps
column 1024 learnt 43 steps
column 832 learnt 44 steps
column 1310 learnt 48 steps
column 923 learnt 68 steps
column 1123 learnt 50 steps
column 1311 learnt 27 steps
column 1312 learnt 13 steps
column 1403 learnt 1000 steps
column 1313 learnt 22 steps
column 833 learnt 68 steps
column 924 learnt 44 steps
column 1212 learnt 1000 steps
column 1314 learnt 188 steps
column 925 learnt 41 steps
column 834 learnt 104 st

column 1521 learnt 1000 steps
column 871 learnt 1000 steps
column 1429 learnt 446 steps
column 1058 learnt 38 steps
column 1155 learnt 82 steps
column 1245 learnt 49 steps
column 967 learnt 548 steps
column 1430 learnt 45 steps
column 1156 learnt 37 steps
column 968 learnt 42 steps
column 1059 learnt 50 steps
column 1157 learnt 22 steps
column 872 learnt 60 steps
column 1246 learnt 47 steps
column 1158 learnt 43 steps
column 969 learnt 41 steps
column 1247 learnt 80 steps
column 1522 learnt 596 steps
column 1523 learnt 27 steps
column 1353 learnt 707 steps
column 1159 learnt 40 steps
column 970 learnt 45 steps
column 1060 learnt 152 steps
column 873 learnt 102 steps
column 1061 learnt 35 steps
column 1354 learnt 38 steps
column 1432 learnt 647 steps
column 874 learnt 50 steps
column 1160 learnt 306 steps
column 1355 learnt 125 steps
column 1062 learnt 60 steps
column 875 learnt 50 steps
column 1248 learnt 586 steps
column 1433 learnt 159 steps
column 971 learnt 110 steps
column 1063 le

column 1608 learnt 148 steps
column 1293 learnt 138 steps
column 1552 learnt 1000 steps
column 1553 learnt 15 steps
column 1384 learnt 150 steps
column 1294 learnt 39 steps
column 1295 learnt 48 steps
column 1554 learnt 550 steps
column 1609 learnt 440 steps
column 1385 learnt 220 steps
column 1555 learnt 31 steps
column 1556 learnt 16 steps
column 1456 learnt 960 steps
column 1386 learnt 269 steps
column 1457 learnt 127 steps
column 1458 learnt 95 steps
column 1296 learnt 1000 steps
column 1459 learnt 58 steps
column 1297 learnt 36 steps
column 1387 learnt 148 steps
column 1610 learnt 1000 steps
column 1460 learnt 46 steps
column 1557 learnt 1000 steps
column 1558 learnt 29 steps
column 1388 learnt 521 steps
column 1389 learnt 61 steps
column 1559 learnt 300 steps
column 1613 learnt 687 steps
column 1614 learnt 17 steps
column 1560 learnt 74 steps
column 1461 learnt 529 steps
column 1561 learnt 27 steps
column 1390 learnt 194 steps
column 1298 learnt 1000 steps
column 1562 learnt 20 s